In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import random 
import copy
from scipy.linalg import svd, diagsvd

# Projeto: O Desafio NetFlix
Outros datasets: https://github.com/caserec/Datasets-for-Recommender-Systems

Um problema que existe hoje em dia com as grandes empresas de streaming (Netflix, Spotify, etc.) é que elas têm um acervo de conteúdo muito grande, e os usuários tendem a gostar, cada um, de uma pequena parte desse acervo. Então, como poderíamos escolher quais filmes vão aparecer tela inicial do seu streaming?

O objetivo deste projeto é fazer um sistema que toma essa decisão.

## Quais dados temos à disposição?

Neste projeto, trabalharemos com o [The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset), que tem, entre outras coisas, a avaliação de usuários em relação a filmes. Essa avaliação está na tabela `ratings.csv` - mas, opcionalmente, pode ser usada a `ratings_small.csv`, que tem somente um subconjunto desses dados.

In [ ]:
import pandas as pd
df = pd.read_csv('ratings_small.csv')

Veja que esse dataframe pode facilmente ser transformado numa matriz $A$ que tem uma linha para cada usuário (identificado por `userId`) e uma coluna para cada filme (identificado por `movieId`). O conteúdo da matriz é o *rating* que o usuário atribuiu ao filme. Podemos ignorar a coluna `timestamp`.


In [ ]:
df_matrix = df.to_numpy()


## Desafio: eu vou gostar deste filme?

O que gostaríamos de saber é qual nota um usuário deveria atribuir a um filme que ele ainda não assistiu. Para isso, o procedimento será o seguinte.

1. Vamos escolher aleatoriamente um dos elementos da matriz $A$ e atribuir a ele um valor aleatório, gerando a matriz $B$.
1. O sistema receberá como entrada a matriz $B$ e a posição $i,j$ do valor aleatório. Neste momento, ele não teve acesso à matriz $A$, e, portanto, não tem como saber qual é o valor "real".
1. O sistema deverá retornar o valor real que estava na matriz $A$.
1. Esse procedimento deverá ser repetido várias vezes, de forma a gerar um histograma dos erros cometidos.


In [ ]:
def randomiza_item_matriz(A):
    B = copy.deepcopy(A)
    r_i = random.randint(0,len(A)-1)
    r_j = random.randint(0,len(A[r_i])-1)
    r_v = random.randint(0,11)
    B[r_i,r_j] = r_v
    return B, (r_i,r_j)


## Como o sistema funciona?

A ideia do sistema de recomendação é que existem "perfis" típicos de usuários. Os perfis, para este problema, são vetores que mostram que notas são tipicamente atribuídas para cada filme por usuários daquele perfil. Por exemplo, talvez tenhamos dois perfis e três filmes, e nesse caso poderíamos ter os perfis:

* $p_0 = [2, 5, 2]$, isto é, o perfil $0$ é de uma pessoa que gosta muito do filme $f_1$, e
* $p_1 = [5, 0, 4]$, isto é, o perfil $1$ é de uma pessoa que gosta dos filmes $f_0$ e $f_2$. 

Porém, sabemos que usuários reais raramente se comportam estritamente como um perfil. As notas realmente atribuídas por um usuário aos filmes, então, são modeladas como combinações lineares dos perfis. Por exemplo, podemos ter usuários:

* $u_0 = 0.1 p_0 + 0.9 p_1$, para um usuário muito próximo de $p_1$ mas distante de $p_0$,
* $u_1 = 0.1 p_0 + 0.1 p_1$, para um usuário distante tanto de $p_0$ quanto de $p_1$,

e assim por diante.

Então, o que precisamos é de uma maneira de mapear usuários para perfis, e então perfis para filmes. Precisamos então *decompor* nossa matriz $A$ de usuários $\times$ filmes em componentes:

$
A = X Y Z,
$

onde:
* $A$ tem uma linha por usuário e uma coluna por filme.

In [ ]:
df_ = pd.pivot_table(data = df, index = 'userId', columns ='movieId', values = 'rating',aggfunc='mean')
df_ = df_.fillna(0)
df_.shape

In [ ]:
# df_.iloc[2][44191] teste

* $X$ tem uma linha por usuário e uma coluna por perfil,

In [ ]:
X, Y, Z = np.linalg.svd(df_)
X

* $Y$ é quadrada e mapeia perfis para perfis,

In [ ]:
Y

* $Z$ tem uma linha por perfil e uma coluna por filme.

In [ ]:
Z


Isso se parece bastante com algo que já fizemos nesta aula!

Em nosso teste, ao aleatorizarmos um elemento da matriz $A$, estamos inserindo ruído. Como poderíamos remover esse ruído?

## Descrição do projeto

Neste projeto, o grupo deverá fazer um sistema preditor de nota de filmes por usuário, que funciona nas condições que foram citadas no enunciado (temos conhecimento de todo o dataset, exceto do par filme-usuário específico). O sistema projetado deve ser avaliado usando um histograma dos erros ao longo de várias estimativas. O número de estimações deve ser, no mínimo, mil. O projeto deve ser colocado em um repositório GitHub específico.

Anotações importantes:

1. O grupo deve enviar um link para o repositório GitHub onde está localizada a biblioteca.
2. No diretório principal do repositório, deve haver um programa `demo.py`, que, quando executado, executa todos os testes que geram o histograma de resultados.
3. Como o objetivo do projeto é exatamente implementar o sistema de recomendação, não é permitido usar bibliotecas que fazem recomendações. Toda a parte de algoritmos e álgebra linear deve ser feita pelo próprio grupo usando Numpy ou Scipy.

**ENTREGAS**
* Link para o repositório onde está a biblioteca.
* No `README.md` do repositório, deve haver uma discussão sobre como o sistema funciona. Essa discussão deve corresponder ao que foi feito no código.
* Inclua também, no próprio `README.md`, instruções sobre como rodar o `demo.py` e como usar suas funcionalidades.
* O `README.md` também deve ter uma discussão dos resultados encontrados, incluindo o histograma dos erros e uma conclusão, baseada em dados, sobre se o grupo acredita que o sistema proposto poderia ser usado em produção.

**RUBRICA**

O projeto será avaliado usando três rubricas. [Duas delas dizem respeito à redação e ao código apresentado](rubricas.md), e são compartilhados por todos os projetos. Este projeto tem ainda requisitos específicos que estão na rubrica abaixo. Os níveis são cumulativos, isto é, para passar de um nível, *todos* os requisitos dele devem ser cumpridos. A nota final é baseada na rubrica em que o trabalho obtiver o *menor* desempenho. As rubricas foram inspiradas nos níveis da [Taxonomia de Bloom](https://cft.vanderbilt.edu/guides-sub-pages/blooms-taxonomy/).

| Nível | Descrição | [Tax. de Bloom](https://cft.vanderbilt.edu/guides-sub-pages/blooms-taxonomy/) |
| --- | --- | --- |
| F | Não entregue, entregue sem completar o `README.md`, entregue sem o `demo.py`, entregue sem resolver o problema que foi proposto, ou usando bibliotecas que resolvem o problema central do enunciado.  | Não fez |
| D | Entregue, mas o `README.md` não indica como instalar ou rodar o programa, ou o programa usa a matriz $A$ (sem ruído) como entrada ao invés da matriz $B$. | Entender |
| C | Devido a um erro de implementação ou de teoria, o sistema não retorna um valor coerente para o elemento faltante, isto é, simplesmente retorna um valor qualquer, aleatório, ou calculado sem base teórica adequada. | Compreender |
| B | O método foi aplicado corretamente. | Aplicar |
| A | Além da aplicação correta do método, o grupo realizou um teste de stress (devidamente documentado no README.md), verificando como o histograma de erro se comporta se mais dados da matriz de entrada forem "estragados". | Analisar |

# 